<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/toNamedTuple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/dotmap.ipynb
!pip install importnb
from importnb import Notebook
with Notebook(): 
    from dotmap import DotMap
from collections import namedtuple, UserList
import jax
import numpy as np
import jax.numpy as jnp
import jax.tree_util as tu
from functools import partial
from copy import deepcopy

In [17]:
def recurse(d,e={},root='r'):
    for k,v in d.items():
        if isinstance(v,dict):
            e[k]={}
            e[k]=recurse(v,e[k],root+"_"+k)
        else:
            e[k]=v
    NT=namedtuple(root,e.keys())
    return NT(**e)

In [19]:
def toNamedTuple(orig):
    d=deepcopy(orig)
    stack=[(d,dictHandler(d,'toNamedTupleResult'))]


    while stack:
        nodict=True
        source,setter=stack[-1]
        for k,v in source.items():
            if isinstance(v,dict):
                stack.append((v, dictHandler(source,k)))
                nodict=False
        if nodict:            
            setter(source)
            stack.pop()
            
    return d['toNamedTupleResult']

In [20]:
f=dict(x=2, z=dict(y=3,k=dict(m=2,n=1),a=dict(c=4,b=3)))
g=dict(x=3,y=2, z=dict(a=2))
toNamedTuple(f)

_(x=2, z=_(y=3, k=_(m=2, n=1), a=_(c=4, b=3)))

In [82]:
def printlist(l):
    for item in l:
        print(item)


def dictHandler(build,k):
    def dictSetter(v):
        if isinstance(v,dict):
            NT = namedtuple('_', v.keys())
            build[k]=NT(**v)
        elif isinstance(v,list):
            build[k]=UserList(v)
        return
    return dictSetter

def listHandler(l,i):
    def listSetter(v):
        if isinstance(v,list):
            l[i]=UserList(v)
            # l[i]=UserList(v)
        elif isinstance(v,dict):
            NT = namedtuple('_', v.keys())
            l[i]=NT(**v)
        else:
            l[i]=v
        return
    return listSetter

def listRoot(d,v):
    d=UserList(v)


def toNamedTuple3(orig):
    d=deepcopy(orig)
    if type(d) in (list,tuple):
        stack=[(d, partial(listRoot,d))]
    elif isinstance(d, dict):
        stack=[(d, dictHandler(d,'result'))]

    count=0
    while stack:
        nested=False
        source,setter=stack[-1]
        # printlist(stack)
        # print(source)
        # print(d)
        # print('---')
        if type(source) in (list,tuple):
            for i,v in enumerate(source):
                if type(v) in (dict,list,tuple):
                    stack.append((v, listHandler(source,i)))
                    nested=True

        elif isinstance(source,dict):
            for k,v in source.items():
                if type(v) in (dict,list,tuple):
                    stack.append((v, dictHandler(source,k)))
                    nested=True

        if not nested: 
            setter(source)
            stack.pop()
        count+=1
    return d

In [84]:
d=dict(b=dict(m=2,n=4))
e=dict(c=1,g=dict(z=2))
f=dict(x=3.,y=[d,d])

z=[d,[e,f]]

dt=toNamedTuple3(z)
dt

[{'b': {'m': 2, 'n': 4}}, [{'c': 1, 'g': {'z': 2}}, {'x': 3.0, 'y': [{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]}]]
[{'b': {'m': 2, 'n': 4}}, [{'c': 1, 'g': {'z': 2}}, {'x': 3.0, 'y': [{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]}]]
---
[{'c': 1, 'g': {'z': 2}}, {'x': 3.0, 'y': [{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]}]
[{'b': {'m': 2, 'n': 4}}, [{'c': 1, 'g': {'z': 2}}, {'x': 3.0, 'y': [{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]}]]
---
{'x': 3.0, 'y': [{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]}
[{'b': {'m': 2, 'n': 4}}, [{'c': 1, 'g': {'z': 2}}, {'x': 3.0, 'y': [{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]}]]
---
[{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]
[{'b': {'m': 2, 'n': 4}}, [{'c': 1, 'g': {'z': 2}}, {'x': 3.0, 'y': [{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]}]]
---
{'b': {'m': 2, 'n': 4}}
[{'b': {'m': 2, 'n': 4}}, [{'c': 1, 'g': {'z': 2}}, {'x': 3.0, 'y': [{'b': {'m': 2, 'n': 4}}, {'b': {'m': 2, 'n': 4}}]}]]
---
{'m': 2, '

[_(b=_(m=2, n=4)),
 [_(c=1, g=_(z=2)), _(x=3.0, y=[_(b=_(m=2, n=4)), _(b=_(m=2, n=4))])]]

In [72]:
dt[1][1].x

3.0

In [59]:
                pval,tree = tu.tree_flatten(dt)

In [24]:
val,tree

([1, 2, 3.0, 1.0], PyTreeDef([*, [*, {'x': *, 'y': *}]]))

In [25]:
val2 = np.random.normal(size=len(val))

In [26]:
%%timeit
tu.tree_unflatten(tree, val2)

The slowest run took 14.39 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 2.07 µs per loop


In [27]:
ll=UserList([24,5])

In [28]:
ll.data

[24, 5]

In [29]:
ll[0]=6

In [30]:
isinstance(ll,list)

False